## Step 1: Hello, Data!

In [7]:
import pandas as pd

# Load data
primary_df = pd.read_csv("data/Primary.csv")


# Display first 3 rows of the primary dataset
primary_df.head(3)


,Customer_ID,Age,Gender,Income_Level,Marital_Status,Education_Level,Occupation,Location,Purchase_Category,Purchase_Amount,...,Customer_Satisfaction,Engagement_with_Ads,Device_Used_for_Shopping,Payment_Method,Time_of_Purchase,Discount_Used,Customer_Loyalty_Program_Member,Purchase_Intent,Shipping_Preference,Time_to_Decision
0,37-611-6911,22.0,Female,Middle,Married,Bachelor's,Middle,Évry,Gardening & Outdoors,$333.80,...,7.0,NaN,Tablet,Credit Card,3/1/2024,True,False,Need-based,No Preference,2.0
1,29-392-9296,49.0,Male,High,Married,High School,High,Huocheng,Food & Beverages,$222.22,...,5.0,High,Tablet,PayPal,4/16/2024,True,False,Wants-based,Standard,6.0
2,84-649-5117,24.0,Female,Middle,Single,Master's,High,Huzhen,Office Supplies,$426.22,...,7.0,Low,Smartphone,Debit Card,3/15/2024,True,True,Impulsive,No Preference,3.0


## Step 2: Pick the Right Container
I decided to use a class here because it allowed me to group all transaction-related fields and behavior in one place. This made the later steps like cleaning and computing totals easier.



## Step 3: Transaction Class

In [8]:
class Transaction:
    def __init__(self, Customer_ID, Age, Gender, Income_Level, Marital_Status, Education_Level,
                 Occupation, Location, Purchase_Category, Purchase_Amount, **kwargs):
        self.customer_id = Customer_ID
        self.age = Age
        self.gender = Gender
        self.income = Income_Level
        self.status = Marital_Status
        self.education = Education_Level
        self.occupation = Occupation
        self.city = Location
        self.category = Purchase_Category
        
       

    def total(self):
        return self.amount

    def clean(self):
        self.city = self.city.strip().title() if self.city else "Unknown"


## Step 4: Bulk Loader

In [9]:
from typing import List

def load_transactions(filepath: str) -> List[Transaction]:
    import csv
    transactions = []
    with open(filepath, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            t = Transaction(**row)
            
    return transactions

transactions = load_transactions("data/Primary.csv")


## Step 5: Quick Profiling

In [10]:
prices = [t.amount for t in transactions]
print("Unique cities:", len(set([t.city for t in transactions])))


Unique cities: 0


## Step 6: Spot the Grime

Examples of dirty data:
1. Inconsistent capitalization in city names
2. Purchase amount strings with dollar signs
3. Extra whitespace around string fields



In [11]:
## Step 7: Cleaning Rules

In [ ]:
print("Before:", transactions[0].city)
print("After:", transactions[0].city) 
for t in transactions:
    t.clean()



IndexError: list index out of range